In [1]:
import sys, time, pickle
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql.types import *
from IPython.display import display, HTML
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor, RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, CrossValidatorModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import ClusteringEvaluator
import pandas as pd
import numpy as np

from sklearn.feature_selection import mutual_info_regression

import matplotlib.pyplot as plt

# make matplotlib plot sizes larger
plt.rcParams['figure.figsize'] = [30, 20]

conf = SparkConf().setAppName('Steam Random Forest Regressor').setMaster('spark://sparkmaster:7077')
SparkContext.setSystemProperty('spark.executor.memory', '2g') # memory per executor
SparkContext.setSystemProperty('spark.executor.cores', '6') # cores per executor
SparkContext.setSystemProperty('spark.executor.instances', '3') # per worker (computer)

# https://spark.apache.org/docs/3.0.0-preview/configuration.html#dynamic-allocation
# https://stackoverflow.com/questions/26168254/how-to-set-amount-of-spark-executors
# https://blog.cloudera.com/how-to-tune-your-apache-spark-jobs-part-2/

# SparkContext.setSystemProperty("spark.shuffle.service.enabled", "True") # required for dynamic allocation below
# SparkContext.setSystemProperty("spark.dynamicAllocation.enabled", "True")
# SparkContext.setSystemProperty("spark.executor.cores", "4")
# SparkContext.setSystemProperty("spark.dynamicAllocation.minExecutors", "1")
# SparkContext.setSystemProperty("spark.dynamicAllocation.maxExecutors", "5")
# SparkContext.setSystemProperty('spark.executor.memory', '2g') # memory per executor

sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [2]:
sc._conf.getAll()

[('spark.executor.memory', '2g'),
 ('spark.driver.host', 'jupyterlab'),
 ('spark.executor.instances', '3'),
 ('spark.app.name', 'Steam Random Forest Regressor'),
 ('spark.app.id', 'app-20210726204145-0005'),
 ('spark.executor.id', 'driver'),
 ('spark.master', 'spark://sparkmaster:7077'),
 ('spark.app.startTime', '1627353704369'),
 ('spark.executor.cores', '6'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '32991'),
 ('spark.ui.showConsoleProgress', 'true')]

In [3]:
# https://stackoverflow.com/questions/37513355/converting-pandas-dataframe-into-spark-dataframe-error
schema = StructType([ StructField("type", StringType(), True)\
                       ,StructField("name", StringType(), True)\
                       ,StructField("required_age", IntegerType(), True)\
                       ,StructField("appid", IntegerType(), True)\
                       ,StructField("release_date", DateType(), True)\
                       ,StructField("initial_price", DoubleType(), True)\
                       ,StructField("metacritic_score", DoubleType(), True)\
                       ,StructField("windows", BooleanType(), True)\
                       ,StructField("mac", BooleanType(), True)\
                       ,StructField("linux", BooleanType(), True)\
                       ,StructField("publisher", StringType(), True)\
                       ,StructField("developer", StringType(), True)\
                       ,StructField("number_dlc", IntegerType(), True)\
                       ,StructField("number_genres", IntegerType(), True)\
                       ,StructField("number_categories", IntegerType(), True)\
                       ,StructField("total_recommendations", DoubleType(), True)\
                       ,StructField("release_day_in_year", DoubleType(), True)\
                       ,StructField("sum_recommendations_up", DoubleType(), True)\
                       ,StructField("sum_recommendations_down", DoubleType(), True)\
                       ,StructField("date", DateType(), True)\
                       ,StructField("days_until_discount", DoubleType(), True)\
                       ,StructField("start_players_-1", DoubleType(), True)\
                       ,StructField("start_players_0", DoubleType(), True)\
                       ,StructField("start_players_1", DoubleType(), True)\
                       ,StructField("start_players_2", DoubleType(), True)\
                       ,StructField("start_players_3", DoubleType(), True)\
                       ,StructField("start_players_4", DoubleType(), True)\
                       ,StructField("start_players_5", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_-1", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_0", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_1", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_2", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_3", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_4", DoubleType(), True)\
                       ,StructField("start_twitch_viewers_5", DoubleType(), True)\
                       ,StructField("start_twitch_rank_-1", DoubleType(), True)\
                       ,StructField("start_twitch_rank_0", DoubleType(), True)\
                       ,StructField("start_twitch_rank_1", DoubleType(), True)\
                       ,StructField("start_twitch_rank_2", DoubleType(), True)\
                       ,StructField("start_twitch_rank_3", DoubleType(), True)\
                       ,StructField("start_twitch_rank_4", DoubleType(), True)\
                       ,StructField("start_twitch_rank_5", DoubleType(), True)\
                       ,StructField("hasLootBoxes", StringType(), True)\
                       ,StructField("isMajorTitle", StringType(), True)\
                       ,StructField("medianScore", DoubleType(), True)\
                       ,StructField("numReviews", DoubleType(), True)\
                       ,StructField("numTopCriticReviews", DoubleType(), True)\
                       ,StructField("percentRecommended", DoubleType(), True)\
                       ,StructField("percentile", DoubleType(), True)\
                       ,StructField("tier", StringType(), True)\
                       ,StructField("topCriticScore", DoubleType(), True)\
                       ,StructField("game_type", StringType(), True)\
                       ,StructField("number_platforms", DoubleType(), True)\
                       ,StructField("number_skus", DoubleType(), True)\
                       ,StructField("type_num", IntegerType(), True)\
                       ,StructField("hasLootBoxes_num", IntegerType(), True)\
                       ,StructField("isMajorTitle_num", IntegerType(), True)\
                       ,StructField("tier_num", IntegerType(), True)\
                       ,StructField("game_type_num", IntegerType(), True)\
                       ,StructField("windows_num", IntegerType(), True)\
                       ,StructField("mac_num", IntegerType(), True)\
                       ,StructField("linux_num", IntegerType(), True)\
                       ,StructField("developer_num", IntegerType(), True)\
                       ,StructField("publisher_num", IntegerType(), True)\
                       ,StructField("cluster_prediction", IntegerType(), True)])

In [4]:
testData = sqlContext.read.load("/work/testData", schema=schema)
trainingData = sqlContext.read.load("/work/trainingData", schema=schema)

In [5]:
testData.summary()

DataFrame[summary: string, type: string, name: string, required_age: string, appid: string, initial_price: string, metacritic_score: string, publisher: string, developer: string, number_dlc: string, number_genres: string, number_categories: string, total_recommendations: string, release_day_in_year: string, sum_recommendations_up: string, sum_recommendations_down: string, days_until_discount: string, start_players_-1: string, start_players_0: string, start_players_1: string, start_players_2: string, start_players_3: string, start_players_4: string, start_players_5: string, start_twitch_viewers_-1: string, start_twitch_viewers_0: string, start_twitch_viewers_1: string, start_twitch_viewers_2: string, start_twitch_viewers_3: string, start_twitch_viewers_4: string, start_twitch_viewers_5: string, start_twitch_rank_-1: string, start_twitch_rank_0: string, start_twitch_rank_1: string, start_twitch_rank_2: string, start_twitch_rank_3: string, start_twitch_rank_4: string, start_twitch_rank_5:

In [6]:
trainingData.summary()

DataFrame[summary: string, type: string, name: string, required_age: string, appid: string, initial_price: string, metacritic_score: string, publisher: string, developer: string, number_dlc: string, number_genres: string, number_categories: string, total_recommendations: string, release_day_in_year: string, sum_recommendations_up: string, sum_recommendations_down: string, days_until_discount: string, start_players_-1: string, start_players_0: string, start_players_1: string, start_players_2: string, start_players_3: string, start_players_4: string, start_players_5: string, start_twitch_viewers_-1: string, start_twitch_viewers_0: string, start_twitch_viewers_1: string, start_twitch_viewers_2: string, start_twitch_viewers_3: string, start_twitch_viewers_4: string, start_twitch_viewers_5: string, start_twitch_rank_-1: string, start_twitch_rank_0: string, start_twitch_rank_1: string, start_twitch_rank_2: string, start_twitch_rank_3: string, start_twitch_rank_4: string, start_twitch_rank_5:

In [7]:
trainingData.head()

In [8]:
# load models in from the previous step
rf_model = RandomForestRegressor.load("/work/steam-randomforest-model/")
#gbt_model = CrossValidatorModel.load("/work/steam-gbt-model/")
#linear_model = CrossValidatorModel.load("/work/steam-linear-model/")

Py4JJavaError: An error occurred while calling o71.load.
: java.lang.UnsupportedOperationException: empty collection
	at org.apache.spark.rdd.RDD.$anonfun$first$1(RDD.scala:1465)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:414)
	at org.apache.spark.rdd.RDD.first(RDD.scala:1463)
	at org.apache.spark.ml.util.DefaultParamsReader$.loadMetadata(ReadWrite.scala:587)
	at org.apache.spark.ml.util.DefaultParamsReader.load(ReadWrite.scala:465)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.i

In [ ]:
predictions = rf_model.transform(testData)
gbt_predictions = gbt_model.transform(testData)
linear_predictions = linear_model.transform(testData)

# Random Forest Results

In [ ]:
evaluator = RegressionEvaluator(labelCol="days_until_discount", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("RMSE: " + str(rmse))

evaluator = RegressionEvaluator(labelCol="days_until_discount", predictionCol="prediction", metricName="mae")

mae = evaluator.evaluate(predictions)
print("MAE: " + str(mae))

evaluator = RegressionEvaluator(labelCol="days_until_discount", predictionCol="prediction", metricName="r2")

r2 = evaluator.evaluate(predictions)
print("r2: " + str(r2))

rfPred = rf_model.transform(sqlContext.createDataFrame(df))

rfResult = rfPred.toPandas()

In [ ]:
rfResult

In [ ]:
plt.plot(rfResult.days_until_discount, rfResult.prediction, 'bo')
plt.xlabel('Actual days until %d%% off' % (percentage_discount_predict))
plt.ylabel('Predicted days')
plt.suptitle("Random Forest Model Performance (RMSE: %f, MAE: %f, R2: %f)" % (rmse, mae, r2))
plt.show()

In [ ]:
bestPipeline = rf_model.bestModel
bestModel = bestPipeline.stages[1]

importances = bestModel.featureImportances

x_values = list(range(len(importances)))

plt.bar(x_values, importances, orientation = 'vertical')
plt.xticks(x_values, feature_list, rotation=90)
plt.ylabel('Importance')
plt.xlabel('Feature')
plt.title('Feature Importance')

In [ ]:
print("Best hyperparameters")
print('numTrees - ', bestModel.getNumTrees)
print('maxDepth - ', bestModel.getOrDefault('maxDepth'))

# Gradient Boosted Tree Results

In [ ]:
evaluator = RegressionEvaluator(labelCol="days_until_discount", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(gbt_predictions)
print("RMSE: " + str(rmse))

evaluator = RegressionEvaluator(labelCol="days_until_discount", predictionCol="prediction", metricName="mae")

mae = evaluator.evaluate(gbt_predictions)
print("MAE: " + str(mae))

evaluator = RegressionEvaluator(labelCol="days_until_discount", predictionCol="prediction", metricName="r2")

r2 = evaluator.evaluate(gbt_predictions)
print("r2: " + str(r2))

rfPred = gbt_model.transform(sqlContext.createDataFrame(df))

rfResult = rfPred.toPandas()

In [ ]:
plt.plot(rfResult.days_until_discount, rfResult.prediction, 'bo')
plt.xlabel('Actual days until %d%% off' % (percentage_discount_predict))
plt.ylabel('Predicted days')
plt.suptitle("GBT Model Performance (RMSE: %f, MAE: %f, R2: %f)" % (rmse, mae, r2))
plt.show()

In [ ]:
bestPipeline = gbt_model.bestModel
bestModel = bestPipeline.stages[1]

importances = bestModel.featureImportances

x_values = list(range(len(importances)))

plt.bar(x_values, importances, orientation = 'vertical')
plt.xticks(x_values, feature_list, rotation=90)
plt.ylabel('Importance')
plt.xlabel('Feature')
plt.title('Feature Importance')

In [ ]:
print("Best hyperparameters")
print('numTrees - ', bestModel.getNumTrees)
print('maxDepth - ', bestModel.getOrDefault('maxDepth'))
print('maxIter - ', bestModel.getOrDefault('maxIter'))

# Linear Regression Results

In [ ]:
evaluator = RegressionEvaluator(labelCol="days_until_discount", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(linear_predictions)
print("RMSE: " + str(rmse))

evaluator = RegressionEvaluator(labelCol="days_until_discount", predictionCol="prediction", metricName="mae")

mae = evaluator.evaluate(linear_predictions)
print("MAE: " + str(mae))

evaluator = RegressionEvaluator(labelCol="days_until_discount", predictionCol="prediction", metricName="r2")

r2 = evaluator.evaluate(linear_predictions)
print("r2: " + str(r2))

rfPred = gbt_model.transform(sqlContext.createDataFrame(df))

rfResult = rfPred.toPandas()

In [ ]:
plt.plot(rfResult.days_until_discount, rfResult.prediction, 'bo')
plt.xlabel('Actual days until %d%% off' % (percentage_discount_predict))
plt.ylabel('Predicted days')
plt.suptitle("Linear Model Performance (RMSE: %f, MAE: %f, R2: %f)" % (rmse, mae, r2))
plt.show()

In [ ]:
bestPipeline = linear_model.bestModel
bestModel = bestPipeline.stages[1]

In [ ]:
print("Best hyperparameters")
print('regParam - ', bestModel.getOrDefault('regParam'))
print('fitIntercept - ', bestModel.getOrDefault('fitIntercept'))
print('elasticNetParam - ', bestModel.getOrDefault('elasticNetParam'))

In [ ]:
sc.stop()